# 3.6 手書き数字認識

In [ ]:
import sys,os,os.path
HOME='/home/jovyan/'

In [ ]:
# %cd /content/drive/MyDrive/work/
# !git clone https://github.com/oreilly-japan/deep-learning-from-scratch.git

In [ ]:
# %cd /content/drive/MyDrive/work/deep-learning-from-scratch/ch03/
base_path=HOME + '/work/DL_scratch/' + 'deep-learning-from-scratch/ch03'
%cd $base_path

## 3.6.1. MNISTデータセット

- 訓練画像が60000枚
- テスト画像がが10000枚
- 0から9までの数字画像から構成されている
- 28x28のグレー画像(1chのみ)



In [ ]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

# load_mnistは以下の形式でMNISTデータを返す
# (訓練画像, 訓練ラベル), (テスト画像, テストラベル)
# arg (normalize): 正規化
# arg (flatten): 画像を平滑化する(1次元配列にする)
# 
(x_train, t_train), (x_test, t_test) = load_mnist(
    flatten=True, normalize=False)

print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from dataset.mnist import load_mnist

sys.path.append(os.pardir)

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(
    flatten=True, normalize=False
)

img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)
img = img.reshape(28, 28)   # 平滑化したデータを元画像(28x28)に変形
print(img.shape)

# img_show(img)
plt.imshow(img)

## 3.6.2 ニューラルネットワークの推論処理

MNISTの推論処理を行うニューラルネットは以下のニューロン数で構成される

- 入力層: 784個
- 出力層: 10個
- 隠れ層1: 50個
- 隠れ層2: 100個

- ここで入出力層のニューロン数は以下と対応する
    - 入力層: 入力画像サイズのピクセル数(28x28=784)
    - 出力層: クラス分類の個数(MNISTはは0~9となる10種類の数字)

- 隠れ層のニューロン数は任意の個数に設定できる


In [ ]:
sys.path.append(os.pardir)
import pickle

from dataset.mnist import load_mnist
from common.functions import softmax, sigmoid

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open('sample_weight.pkl', 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    print(f'W2: {W2.shape}')
    print(f'W3: {W3.shape}')

    return y

In [ ]:
# MNISTデータセットの取得
x, t = get_data()
# モデルの定義
network = init_network()
accuracy_cnt = 0

# 推論の実行
for i in range(len(x)):
    y = predict(network, x[i])

    print(f'y: {y}')

    p = np.argmax(y)    # 最も確率の高い要素ののidxを取得
    print(f'p: {p}')


    if p == t[i]:
        accuracy_cnt += 1

    img = x[i]
    img = img.reshape(28, 28)
    plt.imshow(img)
    
    if i > 5:
        break

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

## 正規化と前処理

- データをある決まった範囲に変換する処理を正規化(normalize)と呼ぶ

- NNの入力データに対して何らかの決まった変換を行うことを前処理(pre-processing)と呼ぶ

- 前処理ははNNにおいて実践的によく用いられる
- 前処理の有効性は識別性能の向上が学習の高速化など多くの実験によって示されている

- 上記, MNISTの例では「前処理としてデータ全体の値をを255で割る」単純な正規化を行なった

- 実際の前処理では、データ全体の分布を考慮して前処理を行うことが多い

- たとえば, データ全体の平均や標準偏差を利用する前処理
- データ全体がが0を中心に分布するように移動させる
- データの広がりをある範囲に収める

- データ全体の分布の形状を均一にする白色化(whitening)

## 3.6.3. バッチ処理

ニューラルネットの各層の重みの形状を出力してみる

In [ ]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

print('x.shape: ', x.shape)
print('x[0].shape: ', x[0].shape)
print('W1.shape: ', W1.shape)
print('W2.shape: ', W2.shape)
print('W3.shape: ', W3.shape)

y = predict(network, x[0])
print('y.shape: ', y.shape)

- 784の要素からなる1次元配列が入力されている: -> x[0].shape
- 1次元の配列が出力されている: -> W3.shape

- 100枚の画像をまとめて1回のpredict()で処理することを考える
- これには, xの形状をを100x784として, 100枚分のデータをまとめて入力すればよい.

- その場合, 入出力データの形状は以下のようになる
    - 入力データ形状: 100x784(入力データ個数, ピクセル数)
    - 出力データ形状: 100x10(入力データ個数 * 出力クラス数)

- 出力データは以下のような形で格納されることになる
    - x[0...n]: 0からn番目の画像データ
    - y[0...n]: 0からn番目の画像の推論結果

- まとまりのある入力データを**バッチ(batch)**と呼ぶ
- batchには**束**という意味があり, 画像がお札のように束になっているイメージで考えるとよい

- バッチ処理にはコンピュータで計算する上で大きな利点がある
    - バッチ処理によって, 1枚あたりの処理時間を大幅に短縮できる
    - これは数値計算ライブラリが大きな配列の計算を効率よく処理できるように最適化が行われていることに起因する
    - NNの計算において, データ転送がボトルネックになる場合はバッチ処理を行うことでバス帯域の負荷を軽減できる
    - **大きな配列を一度に計算する方が、分割した小さい配列を逐次処理するより速く計算できる**


In [ ]:
## バッチ処理の実装

x, t = get_data()
network = init_network()

batch_size = 100 # バッチの数
accuracy_cnt = 0
for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print('Accuracy:' + str(float(accuracy_cnt) / len(x)))

解説はあとで読む

- range()
- argmax()で最大値ののindexを取得するする
    - axis=1を指定と1次元目の要素毎に最大値のindexを見つけることを指定している
    - 
    